    # Working with parquet files
    
    ## Objective
    
    + In this assignment, we will use the data downloaded with the module `data_manager` to create features.
    
    (11 pts total)
    
    ## Prerequisites
    
    + This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.
    

+ Load the environment variables using dotenv. (1 pt)

In [135]:
# Write your code below.
#%load_ext dotenv
%reload_ext dotenv
%dotenv ../01_materials/labs/.env

In [136]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [137]:
import os
from glob import glob

directory_path = os.getenv("PRICE_DATA", '../../05_src/data/prices')
parquet_files = glob(f"{directory_path}/**/*.0.parquet", recursive=True)

print(len(parquet_files))

11207


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [138]:
# Write your code below.
import dask.dataframe as dd
dd_px = dd.read_parquet(parquet_files)
print("Number of partitions:", dd_px.npartitions)

Number of partitions: 11207


In [139]:
dd_px = dd_px.repartition(npartitions=30)
dd_px.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463028,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04 00:00:00+00:00,40.142937,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05 00:00:00+00:00,37.652863,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06 00:00:00+00:00,36.219193,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07 00:00:00+00:00,39.237450,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000


In [140]:
dd_px['Close_lag'] = dd_px['Close'].shift(1)
dd_px['Adj_Close_lag'] = dd_px['Adj Close'].shift(1)
dd_px['returns'] = (dd_px['Adj Close'] / dd_px['Adj_Close_lag']) - 1
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

dd_feat=dd_px
dd_feat.head(10)

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463028,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,NaN,8.270744
A,2000-01-04 00:00:00+00:00,40.142937,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,43.463028,-0.076389,2.950645
A,2000-01-05 00:00:00+00:00,37.652863,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,40.142937,-0.062030,4.425964
A,2000-01-06 00:00:00+00:00,36.219193,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,37.652863,-0.038076,2.771820
A,2000-01-07 00:00:00+00:00,39.237450,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,36.219193,0.083333,4.962444
A,2000-01-10 00:00:00+00:00,41.614330,49.311516,49.803291,48.327969,49.356224,2148446,Health Care,Life Sciences Tools & Services,2000,46.494991,39.237450,0.060577,1.475323
A,2000-01-11 00:00:00+00:00,41.048420,48.640915,49.311516,47.523247,49.311516,1855985,Health Care,Life Sciences Tools & Services,2000,49.311516,41.614330,-0.013599,1.788269
A,2000-01-12 00:00:00+00:00,40.218395,47.657368,48.640915,45.824390,48.640915,1429874,Health Care,Life Sciences Tools & Services,2000,48.640915,41.048420,-0.020221,2.816525
A,2000-01-13 00:00:00+00:00,40.822033,48.372677,49.937412,47.210300,48.909157,1134337,Health Care,Life Sciences Tools & Services,2000,47.657368,40.218395,0.015009,2.727112


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [141]:
df_feat = dd_feat.compute()
df_feat.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463028,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,NaN,8.270744
A,2000-01-04 00:00:00+00:00,40.142937,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,43.463028,-0.076389,2.950645
A,2000-01-05 00:00:00+00:00,37.652863,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,40.142937,-0.062030,4.425964
A,2000-01-06 00:00:00+00:00,36.219193,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,37.652863,-0.038076,2.771820
A,2000-01-07 00:00:00+00:00,39.237450,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,36.219193,0.083333,4.962444


In [142]:
def calculate_rolling_avg_return(group):
    return group.rolling(window=10).mean()

df_feat['rolling_avg_return'] = df_feat.groupby('ticker')['returns'].transform(calculate_rolling_avg_return)

print(df_feat[['returns', 'rolling_avg_return']].head(15)) 

Price    returns  rolling_avg_return
ticker                              
A            NaN                 NaN
A      -0.076389                 NaN
A      -0.062030                 NaN
A      -0.038076                 NaN
A       0.083333                 NaN
A       0.060577                 NaN
A      -0.013599                 NaN
A      -0.020221                 NaN
A       0.015009                 NaN
A       0.011091                 NaN
A       0.045704            0.000540
A      -0.020979            0.006081
A      -0.026786            0.009605
A       0.009174            0.014330
A      -0.003636            0.005633


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No, can be computed within Dask dataframe
+ Would it have been better to do it in Dask? Why? 
Depends on the data size. If the data size is larger than the capacity of the computer memory, it is better to use Dask to slice the data into many partitions and take advantage of parallel computing


(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.